<a href="https://colab.research.google.com/github/davidbaines/translate_docx/blob/main/MyDrive_copy_of_Text_Translation_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This Notebook shows a full pipeline for Translation using Facebook's No Language Left Behind (NLLB) model. 

Translate text from our original language to whatever language NLLB supports. 

Source: https://colab.research.google.com/drive/1fsbzykS5ANEMVcn7gtp8Wl7gkmRzbwOW

Webpage: https://medium.com/mlearning-ai/text-translation-using-nllb-and-huggingface-tutorial-7e789e0f7816

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# download the language model pretrained file
!wget https://dl.fbaipublicfiles.com/nllb/lid/lid218e.bin
!pip install fasttext

import fasttext

pretrained_lang_model = "/content/lid218e.bin" # path of pretrained model file
model = fasttext.load_model(pretrained_lang_model)

text = "صباح الخير، الجو جميل اليوم والسماء صافية."
predictions = model.predict(text, k=1) 
print(predictions)
input_lang = predictions[0][0].replace('__label__', '')
print(input_lang)

--2023-02-03 02:34:23--  https://dl.fbaipublicfiles.com/nllb/lid/lid218e.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1176355829 (1.1G) [application/octet-stream]
Saving to: ‘lid218e.bin’

lid218e.bin         100%[===================>]   1.09G  24.4MB/s    in 48s     

2023-02-03 02:35:12 (23.4 MB/s) - ‘lid218e.bin’ saved [1176355829/1176355829]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=4397594 sha256=edb26f3bcef0c3f657c209667e50db3b1adcacbd620e930178614831a

# Downloads and Imports

In [3]:
!pip install -U pip transformers
!pip install sentencepiece
!pip install python-docx
!pip install nltk
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 113.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 83.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import docx
import glob
from pathlib import Path

# Download NLLB model

In [5]:
# Smallest 600M parameter model - distilled
# checkpoint = 'facebook/nllb-200-distilled-600M'

# Medium 1.3B parameter model - distilled
checkpoint = 'facebook/nllb-200-distilled-1.3B'

# Medium 1.3B parameter model
# 1.3B parameter model

# Large 3.3B parameter model
# checkpoint = 'facebook/nllb-200-3.3B'


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Test tokenizer and translation.

In [5]:
#Test sentence tokenizer:
para = "Hello World. It's good to see you. Thanks for buying this book."
sent_tokenize(para)

['Hello World.', "It's good to see you.", 'Thanks for buying this book.']

In [8]:
input_lang = 'eng_Latn'
target_lang = 'spa_Latn'
translation_pipeline = pipeline('translation', 
                                model=model, 
                                tokenizer=tokenizer, 
                                src_lang=input_lang, 
                                tgt_lang=target_lang, 
                                max_length = 400)
output = translation_pipeline(para)
print(output[0]['translation_text'])

Hola mundo, me alegro de verte gracias por comprar este libro


# Define Functions

In [10]:
def get_paragraphs_from_docx(file):
        
    paras = []
    # Open connection to Word Document
    doc = docx.Document(file)
    
    # read in each paragraph in file and store the style name with it.
    for para in doc.paragraphs:
        this_para = {'style': para.style.name}
        sentences = []
        for sentence in sent_tokenize(para.text):
            sentences.append(sentence)
        this_para['sentences'] = sentences

    #print(f'Found {len(styles_in_doc)} styles {styles_in_doc} in this document.')    
    return paras


def translate_docx(file):
    
    paras = []

    # Open connection to Word Document
    doc = docx.Document(file)  

    # read in each paragraph in file and store the style name with it.
    for para in doc.paragraphs:
        this_para = {'style': para.style.name}
        sentences = [sentence for sentence in sent_tokenize(para.text)]
        translations = [translation_pipeline(sentence)[0]['translation_text'] for sentence in sentences]

        this_para['sentences'] = sentences
        this_para['translations'] = translations
        paras.append(this_para)
        
        # This line was a great simplification of the find and replace code.
        para.text = " ".join(translations)

        # I'm not sure this is required, since the style shouldn't have changed.
        para.style = this_para['style']

        
        #print(this_para)

    doc.save(file)
    #print(f'Found {len(styles_in_doc)} styles {styles_in_doc} in this document.')    
    return paras

In [11]:
input_lang = 'eng_Latn'
output_lang = 'tpi_Latn'

input_folder = Path("drive/MyDrive/EIL-Mark")
output_folder = Path("drive/MyDrive/EIL-Mark-Tagalog")
ext_in = 'docx'
ext_out = 'docx'

# Copy the input files to the output directory work only on the copies.
# copy_tree(input_folder, output_folder)


translation_pipeline = pipeline('translation', 
                        model=model, 
                        tokenizer=tokenizer, 
                        src_lang=input_lang, 
                        tgt_lang=output_lang, 
                        max_length = 400)

# Get list of copied files.
output_files = [file for file in output_folder.rglob("*." + ext_in)]
print(f"Found {len(output_files)} {ext_in} files in {output_folder.resolve()}")

Found 321 docx files in /content/drive/MyDrive/EIL-Mark-Tagalog


In [43]:
# Try a version which replaces all the existing text.
#input_files = [output_folder / "0.intro" / "Mark Introduction 1.docx"]
#input_files = [output_folder /"1.1-1.13" / file for file in ["1 - Hear and Heart.docx"]]
#input_files = [output_folder /"1.1-1.13" / file for file in ["2 - Setting the Stage.docx"]]
input_files = [output_folder /"1.1-1.13" / file for file in ["3 - Defining the Scenes.docx", "4 - Embodying the Text.docx", "5 - Filling the Gaps.docx" ]]

input_files = [file for file in output_folder.rglob("*.docx")]
processed_folders = sorted(["10.13-10.31", "1.29-1.34", "10.1-10.12", "0.intro", 
                     "11.1-11.11", "1.14-1.20", "1.1-1.13", "1.35-1.39", 
                     "1.40-1.45", "1.21-1.28", "12.38-12.44", "11.12-11.26", 
                     "12.18-12.27", "12.28-12.34", "12.13-12.17", "13.24-13.31",
                     "12.1-12.12", "12.35-12.37", "13.1-13.8", "13.32-13.37",
                     "14.32-14.42", "13.9-13.23", "15.16-15.32", "14.12-14.26", 
                     "14.53-14.65", "15.1-15.15", "14.66-14.72", "14.1-14.11", 
                     "14.27-14.31", "14.43-14.52", "2.18-2.22", "3.20-3.35", 
                     "3.13-3.19", "15.40-15.47", "16.1-16.8", "15.33-15.39", 
                     "2.23-3.6", "2.13-2.17", "16.9-16.20", "2.1-2.12", 
                     "4.1-4.20", "5.35-5.43", "6.1-6.6a", "4.21-4.25", 
                     "5.21-5.34", "5.1-5.20", "4.26-4.34", "3.7-3.12", 
                     "6.14-6.29", "4.35-4.41", "6.30-6.44", "7.9-7.13"])

processed_folders = [output_folder / processed_folder for processed_folder in processed_folders]                    

print(f"Found {len(input_files)} input files.")
unprocessed_files = [input_file for input_file in input_files if input_file.parent not in processed_folders]
print(f"There are {len(unprocessed_files)} files still to process.")


Found 321 input files.
There are 70 files still to process.


In [50]:
for unprocessed_file in unprocessed_files:
    unprocessed_folders = sorted(set(unprocessed_file.parent for unprocessed_file in unprocessed_files))
for unprocessed_folder in unprocessed_folders:
    print(unprocessed_folder)


drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel
drive/MyDrive/EIL-Mark-Tagalog/7.1-7.8_1.3BModel
drive/MyDrive/EIL-Mark-Tagalog/7.14-7.23_1.3BModel
drive/MyDrive/EIL-Mark-Tagalog/7.24-7.30_1.3BModel
drive/MyDrive/EIL-Mark-Tagalog/7.31-7.37_1.3BModel
drive/MyDrive/EIL-Mark-Tagalog/7.9-7.13_1.3BModel
drive/MyDrive/EIL-Mark-Tagalog/8.1-8.10_1.3BModel
drive/MyDrive/EIL-Mark-Tagalog/8.11-8.21_1.3BModel
drive/MyDrive/EIL-Mark-Tagalog/8.22-8.26_1.3BModel
drive/MyDrive/EIL-Mark-Tagalog/8.27-8.30_1.3BModel
drive/MyDrive/EIL-Mark-Tagalog/8.31-9.1_1.3BModel
drive/MyDrive/EIL-Mark-Tagalog/9.14-9.29_1.3BModel
drive/MyDrive/EIL-Mark-Tagalog/9.2-9.13_1.3BModel
drive/MyDrive/EIL-Mark-Tagalog/9.30-9.50_1.3BModel


In [51]:
unprocessed_files = []

for folder in unprocessed_folders:
    print(f"Looking in {folder}")
    files = [file for file in folder.rglob("*." + "docx")]
    print(f"Found: {files}")
    unprocessed_files.extend(files)


for unprocessed_file in unprocessed_files:
    print(unprocessed_file)


Looking in drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel
Found: [PosixPath('drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/5 - Filling the Gaps.docx'), PosixPath('drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/3 - Defining the Scenes.docx'), PosixPath('drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/2 - Setting the Stage.docx'), PosixPath('drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/4 - Embodying the Text.docx'), PosixPath('drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/1 - Hear and Heart.docx')]
Looking in drive/MyDrive/EIL-Mark-Tagalog/7.1-7.8_1.3BModel
Found: [PosixPath('drive/MyDrive/EIL-Mark-Tagalog/7.1-7.8_1.3BModel/5 - Filling the Gaps.docx'), PosixPath('drive/MyDrive/EIL-Mark-Tagalog/7.1-7.8_1.3BModel/3 - Defining the Scenes.docx'), PosixPath('drive/MyDrive/EIL-Mark-Tagalog/7.1-7.8_1.3BModel/2 - Setting the Stage.docx'), PosixPath('drive/MyDrive/EIL-Mark-Tagalog/7.1-7.8_1.3BModel/4 - Embodying the Text.docx'), PosixPath('drive/MyDrive/EIL-Mark-Tagalog

In [52]:
input_files = unprocessed_files

for input_file in input_files:
    
    print(f"Opening {input_file}")
    # Open the input file as a Word document
    try :
        document = docx.Document(input_file)
    except BadZipFile:
        print(f"BadZipFile Error on opening {input_file}")
        continue
    
    # Get output filename.
    #output_file = input_file.with_suffix(".translated.docx").resolve()
    
    output_file = input_file.resolve()
    
    # Save the file.
    document.save(input_file)

    # From docx help
    #document = Document('existing-document-file.docx')
    #document.save('new-file-name.docx')
    
    # Translate the content
    paragraphs = translate_docx(output_file)
    
    print(f"Saved the translated file to {output_file}")

    #for paragraph in paragraphs:
    #    print(paragraph)


Opening drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/5 - Filling the Gaps.docx
Saved the translated file to /content/drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/5 - Filling the Gaps.docx
Opening drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/3 - Defining the Scenes.docx
Saved the translated file to /content/drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/3 - Defining the Scenes.docx
Opening drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/2 - Setting the Stage.docx
Saved the translated file to /content/drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/2 - Setting the Stage.docx
Opening drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/4 - Embodying the Text.docx
Saved the translated file to /content/drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/4 - Embodying the Text.docx
Opening drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/1 - Hear and Heart.docx
Saved the translated file to /content/drive/MyDrive/EIL-Mark-Tagalog/6.6b-6.13_1.3BModel/1 - Hear and Hea

In [6]:
output_folder = Path("drive/MyDrive/EIL-Mark-Tagalog")
ext = 'docx'
output_files = [file for file in output_folder.rglob("*.docx")]

In [30]:
from collections import Counter
from tqdm import tqdm
#pretrained_lang_model = "/content/lid218e.bin" # path of pretrained model file
#model = fasttext.load_model(pretrained_lang_model)

def get_translated_language(output_files):
    results = {}
    for output_file in tqdm(output_files):

        output_file = output_file.resolve()
        
        #print(f"Opening {output_file}")
        # Open the input file as a Word document
        try :
            document = docx.Document(output_file)
        except BadZipFile:
            print(f"BadZipFile Error on opening {output_file}")
            continue

        
        # From docx help
        #document = Document('existing-document-file.docx')
        #document.save(output_file)
        
        # read in each paragraph in file and store the style name with it.
        paragraphs = [para for para in document.paragraphs]
        #print(paragraphs)

        sentences = [sentence for para in document.paragraphs for sentence in sent_tokenize(para.text)]
        
        # Get 5% of the sentences
        #print(sentences)

        languages = Counter()
        for sentence in sentences:
            predictions = model.predict(sentence, k=1) 
            #print(predictions)
            output_lang = predictions[0][0].replace('__label__', '')
            #print(output_lang)
            languages.update([output_lang])
        #print(languages)
        #print(f"There are {len(sentences)} sentences in {output_file}.")
        results[output_file] = {"languages": languages, "sentence_count": len(sentences)}

    for file, info in results.items():
        print(f"There are {info['sentence_count']} sentences in {str(file)[23:]}    {info['languages'].most_common()}")
        #print(f"{info['languages']}")


Found 321 input files.


100%|██████████| 321/321 [04:21<00:00,  1.23it/s]

There are 85 sentences in EIL-Mark-Tagalog/10.13-10.31/4 - Embodying the Text.docx    [('tpi_Latn', 83), ('vie_Latn', 1), ('lit_Latn', 1)]
There are 38 sentences in EIL-Mark-Tagalog/10.13-10.31/5 - Filling the Gaps.docx    [('tpi_Latn', 37), ('vie_Latn', 1)]
There are 9 sentences in EIL-Mark-Tagalog/10.13-10.31/1 - Hear and Heart.docx    [('tpi_Latn', 9)]
There are 62 sentences in EIL-Mark-Tagalog/10.13-10.31/2 - Setting the Stage.docx    [('tpi_Latn', 60), ('kor_Hang', 1), ('arb_Arab', 1)]
There are 40 sentences in EIL-Mark-Tagalog/10.13-10.31/3 - Defining the Scenes.docx    [('tpi_Latn', 38), ('vie_Latn', 1), ('lit_Latn', 1)]
There are 28 sentences in EIL-Mark-Tagalog/1.29-1.34/5 - Filling the Gaps.docx    [('tpi_Latn', 26), ('vie_Latn', 1), ('eus_Latn', 1)]
There are 9 sentences in EIL-Mark-Tagalog/1.29-1.34/1 - Hear and Heart.docx    [('tpi_Latn', 9)]
There are 37 sentences in EIL-Mark-Tagalog/1.29-1.34/4 - Embodying the Text.docx    [('tpi_Latn', 34), ('vie_Latn', 1), ('lit_Latn',